## Wine data analysis
### We shall be analysing a medium sized wine data set, in order to answer some of the questions we are keen to knowing, since we are a non-wine conosiour. The questions are as follows:

1. What country produces the most highly rated wines?
2. What province in that country produces the highly rated wines?
3. What winery has the highest rated wines in that province?
4. What are the high quality top producers?
5. What wine VARIETY is specifically highly rated?
6. What common description details are in the top rated wines?
7. What are the price differences range between different countries?
8. What 

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('/Users/pauliuskomskis/Desktop/github/data/wine-reviews/first150.csv')

In [3]:
df.head(3)

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,variety,winery
0,0,US,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz
1,1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,NaN,Tinta de Toro,Bodega Carmen Rodríguez
2,2,US,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90.0,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley


- Lets import the second data to see whats up

### It appears the second data set has the individual taster names. Lets leave this data for now, to come back later

In [4]:
df.columns

Index(['Unnamed: 0', 'country', 'description', 'designation', 'points',
       'price', 'province', 'region_1', 'region_2', 'variety', 'winery'],
      dtype='object')

In [5]:
df.drop('Unnamed: 0', axis = 1, inplace=True)

## 1. What country produces the most highly rated wines?

In [6]:
country_filt = df['points'].astype('int').max()
df.loc[country_filt, 'country']

'US'

-A quick filter lets us see from 150,000 rows of data, that US is the country that offers highest rated wines. Strange. I would have thought France, Italy. Live and learn eh.

In [7]:
df['points'] = df['points'].astype('str')

- How many wineries of each country has a score of 100?

In [8]:
df['country'].value_counts()

US                        62397
Italy                     23478
France                    21098
Spain                      8268
Chile                      5816
Argentina                  5631
Portugal                   5322
Australia                  4957
New Zealand                3320
Austria                    3057
Germany                    2452
South Africa               2258
Greece                      884
Israel                      630
Hungary                     231
Canada                      196
Romania                     139
Slovenia                     94
Uruguay                      92
Croatia                      89
Bulgaria                     77
Moldova                      71
Mexico                       63
Turkey                       52
Georgia                      43
Lebanon                      37
Cyprus                       31
Brazil                       25
Macedonia                    16
Serbia                       14
Morocco                      12
England 

In [9]:
country_grp = df.groupby(['country'])
country_grp['points'].max()

country
Albania                   88
Argentina                 97
Australia                 98
Austria                   98
Bosnia and Herzegovina    88
Brazil                    88
Bulgaria                  90
Canada                    93
Chile                     95
China                     82
Croatia                   91
Cyprus                    89
Czech Republic            87
Egypt                     84
England                   94
France                    99
Georgia                   92
Germany                   97
Greece                    92
Hungary                   96
India                     91
Israel                    93
Italy                     99
Japan                     85
Lebanon                   91
Lithuania                 85
Luxembourg                88
Macedonia                 89
Mexico                    92
Moldova                   90
Montenegro                82
Morocco                   93
New Zealand               94
Portugal                  99
Romani

In [10]:
country_grp['points'].value_counts()

country    points
Albania    88          2
Argentina  87        700
           86        678
           85        671
           84        636
                    ... 
Uruguay    80          6
           85          6
           87          6
           88          6
           90          2
Name: points, Length: 455, dtype: int64

In [11]:
max_pts = country_grp['points'].apply(lambda x: x.str.contains('100').sum())
max_pts

country
Albania                    0
Argentina                  0
Australia                  3
Austria                    0
Bosnia and Herzegovina     0
Brazil                     0
Bulgaria                   0
Canada                     0
Chile                      0
China                      0
Croatia                    0
Cyprus                     0
Czech Republic             0
Egypt                      0
England                    0
France                     4
Georgia                    0
Germany                    0
Greece                     0
Hungary                    0
India                      0
Israel                     0
Italy                      7
Japan                      0
Lebanon                    0
Lithuania                  0
Luxembourg                 0
Macedonia                  0
Mexico                     0
Moldova                    0
Montenegro                 0
Morocco                    0
New Zealand                0
Portugal                   0
Romani

- it appears that only Australia, Italy, France and US has wineries that scored a perfect 100. Lets find out more about those perfect wineries

## 2. What province in United States produces most of the highest rated wines?

In [12]:
df['points'] = df['points'].astype('int')

In [13]:
province_filt = df['points'].max('country'=='US')
df.loc[province_filt, 'province']

'California'

In [14]:
df['points'] = df['points'].astype('str')
ca_filt = df['province'] == 'California'
df.loc[ca_filt]['points'].str.contains('100').sum()

7

- So there are 7 entries in California where the points score a 100

### 3. What are the top rated wineries in the California province?

In [15]:
df['points'] = df['points'].astype('int')
winery_filt = (df['points'] > 99) & (df['province'] == 'California')


In [16]:
df.loc[winery_filt]

,country,description,designation,points,price,province,region_1,region_2,variety,winery
89399,US,Tasted in a flight of great and famous Napa wi...,NaN,100,200.0,California,Napa Valley,Napa,Cabernet Sauvignon,Cardinale
92916,US,A perfect score has to be considered in the co...,Hillside Select,100,215.0,California,Stags Leap District,Napa,Cabernet Sauvignon,Shafer
98647,US,"A spectacular Pinot Noir, but a very young one...",Litton Estate Vineyard,100,100.0,California,Russian River Valley,Sonoma,Pinot Noir,Williams Selyem
114272,US,Impossibly aromatic. Hard to imagine greater c...,Red Wine,100,245.0,California,Rutherford,Napa,Cabernet Blend,Sloan
122767,US,"A spectacular Pinot Noir, but a very young one...",Litton Estate Vineyard,100,100.0,California,Russian River Valley,Sonoma,Pinot Noir,Williams Selyem
137099,US,Tasted in a flight of great and famous Napa wi...,NaN,100,200.0,California,Napa Valley,Napa,Cabernet Sauvignon,Cardinale
143522,US,Impossibly aromatic. Hard to imagine greater c...,Red Wine,100,245.0,California,Rutherford,Napa,Cabernet Blend,Sloan


### 4. What are the top rated wineries in all of United States:

In [17]:
winery_filt_us = (df['points'] > 99) & (df['country'] == 'US')
df.loc[winery_filt_us]

,country,description,designation,points,price,province,region_1,region_2,variety,winery
19354,US,"In a stunning lineup of Cayuse Syrahs, the En ...",En Chamberlin Vineyard,100,65.0,Oregon,Walla Walla Valley (OR),Oregon Other,Syrah,Cayuse
84034,US,"In a stunning lineup of Cayuse Syrahs, the En ...",En Chamberlin Vineyard,100,65.0,Oregon,Walla Walla Valley (OR),Oregon Other,Syrah,Cayuse
89399,US,Tasted in a flight of great and famous Napa wi...,NaN,100,200.0,California,Napa Valley,Napa,Cabernet Sauvignon,Cardinale
92916,US,A perfect score has to be considered in the co...,Hillside Select,100,215.0,California,Stags Leap District,Napa,Cabernet Sauvignon,Shafer
98647,US,"A spectacular Pinot Noir, but a very young one...",Litton Estate Vineyard,100,100.0,California,Russian River Valley,Sonoma,Pinot Noir,Williams Selyem
114272,US,Impossibly aromatic. Hard to imagine greater c...,Red Wine,100,245.0,California,Rutherford,Napa,Cabernet Blend,Sloan
119194,US,"In a stunning lineup of Cayuse Syrahs, the En ...",En Chamberlin Vineyard,100,65.0,Oregon,Walla Walla Valley (OR),Oregon Other,Syrah,Cayuse
122767,US,"A spectacular Pinot Noir, but a very young one...",Litton Estate Vineyard,100,100.0,California,Russian River Valley,Sonoma,Pinot Noir,Williams Selyem
137099,US,Tasted in a flight of great and famous Napa wi...,NaN,100,200.0,California,Napa Valley,Napa,Cabernet Sauvignon,Cardinale
143522,US,Impossibly aromatic. Hard to imagine greater c...,Red Wine,100,245.0,California,Rutherford,Napa,Cabernet Blend,Sloan


- as we see, 7 wineries are located in California, 3 are located in Oregon, with a perfect score of 100.

### 5. List the top wine varietys of our top producers in United States and how many of our "Perfect 10" wines do they produce:

In [18]:
df.loc[winery_filt_us, 'variety'].value_counts()

Syrah                 3
Cabernet Sauvignon    3
Pinot Noir            2
Cabernet Blend        2
Name: variety, dtype: int64

### 6. List the top wineries and how many of the top wines, that they make of our "Perfect 10":

In [19]:
df.loc[winery_filt, 'winery'].value_counts()

Sloan              2
Cardinale          2
Williams Selyem    2
Shafer             1
Name: winery, dtype: int64

### 7. What description commonalities does our 'Perfect 10' wine list exhibit between each other if any:

In [20]:
df.loc[winery_filt, 'price']

89399     200.0
92916     215.0
98647     100.0
114272    245.0
122767    100.0
137099    200.0
143522    245.0
Name: price, dtype: float64

In [21]:
df.head()

,country,description,designation,points,price,province,region_1,region_2,variety,winery
0,US,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz
1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,NaN,Tinta de Toro,Bodega Carmen Rodríguez
2,US,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90.0,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley
3,US,"This spent 20 months in 30% new French oak, an...",Reserve,96,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi
4,France,"This is the top wine from La Bégude, named aft...",La Brûlade,95,66.0,Provence,Bandol,NaN,Provence red blend,Domaine de la Bégude


In [22]:

df[['price','points']].corr()

,price,points
price,1.000000,0.459863
points,0.459863,1.000000


In [23]:

%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.style.use('ggplot')

In [24]:
all(isinstance(column, str) for column in df.columns)

True

### Lets identify wineries with wine that costs 500 dollars and points above 98

In [25]:
expensive_filt = (df['price'].astype('str') > '500') & (df['points'].astype('str') >= '99')

In [26]:
df.loc[expensive_filt, 'winery']

19356                   Cayuse
34649          Williams Selyem
42798          Williams Selyem
42799                   Cayuse
54343           Château Palmer
54344          Château d'Yquem
54345     Château Pontet-Canet
81688              Stonestreet
84036                   Cayuse
106483          Château Palmer
106484         Château d'Yquem
106485    Château Pontet-Canet
119196                  Cayuse
131383          Château Palmer
131384         Château d'Yquem
131385    Château Pontet-Canet
142677          Château Latour
Name: winery, dtype: object

### Lets pick the wine with score of equal or above 99 points, price above 500 dollars ([expensive_filt]), and see which - country, province and its region produces.

In [27]:
df.groupby(['country', 'province', 'region_1']).apply(lambda df: df.loc[expensive_filt])

country  \
country province   region_1                                 
France  Bordeaux   Margaux                 54343   France   
                                           106483  France   
                                           131383  France   
                   Pauillac                54345   France   
                                           106485  France   
                                           131385  France   
                                           142677  France   
                   Sauternes               54344   France   
                                           106484  France   
                                           131384  France   
US      California Alexander Valley        81688       US   
                   Sonoma Coast            34649       US   
                                           42798       US   
        Oregon     Walla Walla Valley (OR) 19356       US   
                                           42799       US   
                                           84036       US   
                                           119196      US   

                                                                                         description  \
country province   region_1                                                                            
France  Bordeaux   Margaux                 54343   98-100 Barrel sample. Sumptuous, opulent, with...   
                                           106483  98-100 Barrel sample. Sumptuous, opulent, with...   
                                           131383  98-100 Barrel sample. Sumptuous, opulent, with...   
                   Pauillac                54345   98-100 Barrel sample. A superb wine, with the ...   
                                           106485  98-100 Barrel sample. A superb wine, with the ...   
                                           131385  98-100 Barrel sample. A superb wine, with the ...   
                                           142677  98-100 Barrel Sample. This has to be the wine ...   
                   Sauternes               54344   98-100 Barrel sample. The aromas are so rich a...   
                                           106484  98-100 Barrel sample. The aromas are so rich a...   
                                           131384  98-100 Barrel sample. The aromas are so rich a...   
US      California Alexander Valley        81688   This is Cabernet Sauvignon lifted to greatness...   
                   Sonoma Coast            34649   A stupendous Pinot Noir, showing how beautiful...   
                                           42798   This expresses the greatness of its vintage an...   
        Oregon     Walla Walla Valley (OR) 19356   The only one of the Cayuse Syrahs that is co-f...   
                                           42799   In a vintage that produced the finest overall ...   
                                           84036   The only one of the Cayuse Syrahs that is co-f...   
                                           119196  The only one of the Cayuse Syrahs that is co-f...   

                                                                  designation  \
country province   region_1                                                     
France  Bordeaux   Margaux                 54343                Barrel sample   
                                           106483               Barrel sample   
                                           131383               Barrel sample   
                   Pauillac                54345                Barrel sample   
                                           106485               Barrel sample   
                                           131385               Barrel sample   
                                           142677               Barrel sample   
                   Sauternes               54344                Barrel sample   
                                           106484               Barrel sample   
                      

### Lets find some statistical summary of those places:

In [28]:
stat_filt = df.groupby(['country', 'province', 'region_1']).apply(lambda df: df.loc[expensive_filt])

In [29]:
df.groupby(['country']).price.agg([len, min, max])

,len,min,max
country,,,
Albania,2.0,20.0,20.0
Argentina,5631.0,4.0,250.0
Australia,4957.0,5.0,850.0
Austria,3057.0,8.0,1100.0
Bosnia and Herzegovina,4.0,12.0,13.0
Brazil,25.0,11.0,35.0
Bulgaria,77.0,7.0,28.0
Canada,196.0,12.0,145.0
Chile,5816.0,5.0,400.0


In [30]:
df.dtypes

country         object
description     object
designation     object
points           int64
price          float64
province        object
region_1        object
region_2        object
variety         object
winery          object
dtype: object

In [31]:
df[pd.isnull(df['country'])]

,country,description,designation,points,price,province,region_1,region_2,variety,winery
1133,NaN,Delicate white flowers and a spin of lemon pee...,Askitikos,90,17.0,NaN,NaN,NaN,Assyrtiko,Tsililis
1440,NaN,"A blend of 60% Syrah, 30% Cabernet Sauvignon a...",Shah,90,30.0,NaN,NaN,NaN,Red Blend,Büyülübağ
68226,NaN,"From first sniff to last, the nose never makes...",Piedra Feliz,81,15.0,NaN,NaN,NaN,Pinot Noir,Chilcas
113016,NaN,"From first sniff to last, the nose never makes...",Piedra Feliz,81,15.0,NaN,NaN,NaN,Pinot Noir,Chilcas
135696,NaN,"From first sniff to last, the nose never makes...",Piedra Feliz,81,15.0,NaN,NaN,NaN,Pinot Noir,Chilcas


In [32]:
df.dropna(axis=0, how='all', inplace=True)

In [33]:
df.tail()

,country,description,designation,points,price,province,region_1,region_2,variety,winery
150925,Italy,Many people feel Fiano represents southern Ita...,NaN,91,20.0,Southern Italy,Fiano di Avellino,NaN,White Blend,Feudi di San Gregorio
150926,France,"Offers an intriguing nose with ginger, lime an...",Cuvée Prestige,91,27.0,Champagne,Champagne,NaN,Champagne Blend,H.Germain
150927,Italy,This classic example comes from a cru vineyard...,Terre di Dora,91,20.0,Southern Italy,Fiano di Avellino,NaN,White Blend,Terredora
150928,France,"A perfect salmon shade, with scents of peaches...",Grand Brut Rosé,90,52.0,Champagne,Champagne,NaN,Champagne Blend,Gosset
150929,Italy,More Pinot Grigios should taste like this. A r...,NaN,90,15.0,Northeastern Italy,Alto Adige,NaN,Pinot Grigio,Alois Lageder


### Build a model to Predict which type of wine will receive highest rating